NOMBRES: Jose Miguel

APELLIDOS: Gonzalez y Gonzalez

CARNE: 20335

FECHA: 8/23/2024

**Ejercicio 1**
Con los datos, cálcule PPMI, pero aplicando Lapace Smoothing.

In [ ]:
import numpy as np

In [ ]:
# Renombrar columnas para mayor claridad
data.columns = ["Term"] + list(data.iloc[0, 1:])
data = data[1:]

# Convertir los valores numéricos a enteros para realizar los cálculos
for col in data.columns[1:]:
    data[col] = pd.to_numeric(data[col], errors='coerce').fillna(0).astype(int)

# Calcular las frecuencias totales por palabra (sumando filas y columnas)
total_word_frequency = data.set_index("Term").sum(axis=1) + data.drop(columns=["Term"]).sum(axis=0)

# Calcular el total de todas las coocurrencias
total_cooccurrences = data.drop(columns=["Term"]).values.sum()

# Calcular la probabilidad conjunta y aplicar Laplace Smoothing
laplace_k = 1  # Smoothing factor
vocab_size = len(total_word_frequency)  # Tamaño del vocabulario

# Crear una matriz de probabilidades suavizadas
ppmi_matrix = pd.DataFrame(index=data["Term"], columns=data.columns[1:])

for term1 in ppmi_matrix.index:
    for term2 in ppmi_matrix.columns:
        cooccurrence = data.loc[data["Term"] == term1, term2].values[0]
        # Probabilidad conjunta suavizada
        p_xy = (cooccurrence + laplace_k) / (total_cooccurrences + laplace_k * vocab_size ** 2)
        # Probabilidad marginales suavizadas
        p_x = (total_word_frequency[term1] + laplace_k * vocab_size) / (total_cooccurrences + laplace_k * vocab_size ** 2)
        p_y = (total_word_frequency[term2] + laplace_k * vocab_size) / (total_cooccurrences + laplace_k * vocab_size ** 2)
        
        # Calcular PMI
        pmi = np.log2(p_xy / (p_x * p_y)) if p_x * p_y > 0 else 0
        
        # Calcular PPMI
        ppmi_matrix.loc[term1, term2] = max(pmi, 0)

import ace_tools as tools; tools.display_dataframe_to_user(name="PPMI con Laplace Smoothing", dataframe=ppmi_matrix)


**Ejercicio 2**

POC para crear información de entreno

In [1]:
#Librerías que necesitarán
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras import utils
%load_ext tensorboard
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [5]:
sentece = "The wind Crosses the brown land unheard" #De mis poemas fav. The waste land de T.S Elliot
tokens = #tokenizar
vocab, index = #arreglo vocabulario, empezar indice en 1
vocab["algo importante para que no explote el programa"] = 0

#Llene el arreglo para vocabulario




vocab_size = len(vocab)
example_sequence = [vocab[word] for word in tokens]
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(#rellene esta area como le sea pertinente)


In [6]:
#Resultados
inverse_vocab = {index: token for token, index in vocab.items()}
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(5, 4): (land, brown)
(5, 6): (land, unheard)
(1, 2): (the, wind)
(6, 5): (unheard, land)
(3, 4): (crosses, brown)


In [ ]:
target_word, context_word = positive_skip_grams[0]
# Escoja un numero para muestras negativos (que no pertenence al contexto)
num_ns =

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=  # clase que debe ser muestreada como positiva (que pertenece al contexto)
    num_true=1,
    num_sampled=num_ns,
    unique=True,
    range_max=vocab_size,  # [0,vocab_size]
    seed=SEED,
    name="negative_sampling"
)

In [ ]:
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

In [ ]:
sampling_table = tf.keras.preprocessing.sequence. #funcion que construye la tabla de muestreo en forma de frecuencias
print(sampling_table)

In [ ]:
negative_sampling_candidates = tf.expand_dims(negative_sampling_candidates, 1) #se agrega una dimension para poder concatenar
context = tf.concat([context_class, negative_sampling_candidates], 0)
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = tf.squeeze(target_word)
context = tf.squeeze(context)
label = tf.squeeze(label)

#Listo! asi se prepara la info para entrenar

In [2]:
#Informacion a utilizar:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#Paso 1: cree una funcion que estandarice el texto como ya hemos hecho e incrustela en una capa de Tensorflow (TextVectorization)





In [ ]:
vectorize_layer.adapt(text_ds.batch(1024))

In [ ]:
#Paso 2: Usando la POC de creacion de datos de entreno, defina una funcion para este proceso






In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
sequences = list(text_vector_ds.as_numpy_iterator())

In [ ]:
#El siguiente codigo es el modelo Word2Vec usando lo que ya han hecho. Sin embargo, deben agregar la metrica de similitud que vimos en clase, a la cual deben llamar dots
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)


    # dots: (batch, context)
    return dots

In [ ]:
#Entrenar modelo y definir CategoricalCrossEntropy como funcion de perdida. Mostrar el print de accuracy y loss. Si deciden presentar sus resultados en tensorboard, adjuntar capturas.